<span style="color:CYAN;">II. PRE-PROCESS DATA</span>

In [2]:
import numpy as np
import pandas as pd
import datetime

In [3]:
FANPAGE_PATH = 'insightmatlong' 
file_path = "Data\\" 

In [4]:
def read_data(file_path):
    try:
        missing = [np.nan, 'N/A', 'na', '[]', '[None]']
        data = pd.read_csv(file_path + FANPAGE_PATH + '.csv', na_values=missing)
        return data
    except FileNotFoundError:
        print("File not found.")    
df = read_data(file_path)

In [5]:
# Filter relevant data
relevant_columns = ['post_id', 'post_text', 'shared_text', 'time', 'timestamp', 'image', 'video',
                    'video_id', 'comments', 'shares', 'post_url', 'comments_full', 'reactors', 
                    'reactions', 'reaction_count']
df = df[relevant_columns]

In [6]:
# Separate each reaction
reactions_dict = df['reactions'].apply(lambda x : dict(eval(x)))
reactions_df = reactions_dict.apply(pd.Series)
reactions_df.fillna(0, inplace=True)
reactions_df.astype(int)

df = pd.concat([df, reactions_df], axis=1)

In [23]:
# Format time
df['time'] = df['time'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
df['hour'] = df['time'].apply(lambda x : x.hour)

In [24]:
# Seperate commnents from commnents_full
comment_list = [[],[],[],[]]
comments_full = df['comments_full'].dropna(axis=0)

for post in comments_full:
    for commment_part in post.split(', \''):

        id_it = commment_part.find('commenter_id')
        name_it = commment_part.find('commenter_name')
        text_it = commment_part.find('comment_text')
        time_it = commment_part.find('comment_time')

        if id_it != -1 :
            pos = id_it + len('commenter_id') + 4
            comment_list[0].append(commment_part[pos:-1])
        if name_it != -1:
            pos = name_it + len('commenter_name') + 4
            comment_list[1].append(commment_part[pos:-1])
        if text_it != -1:
            pos = text_it + len('comment_text') + 4
            comment_list[2].append(commment_part[pos:-1])
        if time_it != -1:
            pos = time_it + len('comment_time') + 3
            comment_list[3].append(commment_part[pos:])

comment_df = pd.DataFrame(comment_list, index=['commenter_id','commenter_name',
                                               'comment_text', 'comment_time']).transpose()

comment_df['comment_time'] = comment_df['comment_time'].apply(lambda x : eval(x))
comment_df['comment_time'].apply(lambda x : x.strftime("%Y-%m-%d %H:%M:%S"))
comment_df

,commenter_id,commenter_name,comment_text,comment_time
0,100026226853818,Lê Phương Anh,Kien Pham Thuỳ Trang,2023-11-28 23:42:00
1,100004081241345,Bin Nguyễn,ác =))),2023-11-28 23:14:00
2,100047325651298,Nguyễn Kim Hoàng Anh,Trần Lương nhỏ này gọi con cà ty🥸,2023-11-28 23:01:00
3,100044112216057,Trần Lương,Nguyễn Kim Hoàng Anh con cà ty có pàu,2023-11-28 23:32:00
4,100013708272492,Lương Quốc Bảo,Mai N. Hà con thanh long ruột đỏ 🥰,2023-11-28 23:00:00
...,...,...,...,...
3605,100028124555717,Van Anh Nguyen,cute,2023-11-15 00:00:00
3606,100075643504092,Đào Thị Sơn Trà,Thy Thy,2023-11-15 00:00:00
3607,100027001726868,Xuan Huong Ngo,Bích Phượng Lê,2023-11-15 00:00:00
3608,100005660826545,Trần Thị NhưÝ,Hoàng Tuyên =))))),2023-11-15 00:00:00


In [67]:
# Seperate commnents from commnents_full
reactor_list = [[],[],[]]
reactors = df['reactors'].dropna(axis=0)

for person in reactors:
    for reactor_part in person.split(','):

        name_it = reactor_part.find('name')
        link_it = reactor_part.find('link')
        type_it = reactor_part.find('type')

        if name_it != -1 :
            pos = name_it + len('name') + 4
            reactor_list[0].append(reactor_part[pos:-1])
        if link_it != -1:
            pos = link_it + len('link') + 4
            reactor_list[1].append(reactor_part[pos:-1])
        if type_it != -1:
            pos = type_it + len('type') + 4
            reactor_list[2].append(reactor_part[pos:pos+5])


reactor_df = pd.DataFrame(reactor_list, index=['name','link','type']).transpose()

reactor_df['type'] = reactor_df['type'].apply(
   lambda x : str(x).replace('\'', '')).apply(
   lambda x : str(x).replace('}', ''))

In [69]:
reactor_df

,name,link,type
0,Huy Nguyen Tran Duc,https://facebook.com/profile.php?eav=AfYbvgMJd...,like
1,Kieu Quang,https://facebook.com/profile.php?id=1000957119...,like
2,Trúc Quỳnh,https://facebook.com/profile.php?id=1000956540...,haha
3,Maii Phuong,https://facebook.com/profile.php?id=1000952403...,sad
4,Hương Thu,https://facebook.com/profile.php?id=1000946566...,haha
...,...,...,...
7642,Cao Ngu Yên,https://facebook.com/profile.php?id=1000918954...,haha
7643,Đào Ánh,https://facebook.com/profile.php?id=1000918368...,haha
7644,Chi Quỳnh,https://facebook.com/profile.php?id=1000950058...,like
7645,Lương Bạn Liên Ngẫu,https://facebook.com/profile.php?id=1000919673...,haha


In [ ]:
df.to_csv(file_path + 'clean_' + FANPAGE_PATH + '.csv')
comment_df.to_csv(file_path + 'comments.csv')
reactor_df.to_csv(file_path + 'reactors.csv')